# 05. A/B Testing Simulation: Naive vs Smart

## Objective
To quantify the "Efficiency Gain" of our **Smart (Unmet Demand)** algorithm against a **Population-Weighted Baseline**.
This simulates the impact of investing in *N* chargers under both strategies.

## Scenarios
1.  **Scenario A (Baseline):** Distribute *N* chargers proportional to **Vehicle Density** (Population Weighted). This mimics a standard policy approach.
2.  **Scenario B (Smart):** Use Weighted K-Means on **Unmet_Demand** to place *N* chargers. This targets underserved high-demand areas.

## Metrics
*   **EV Population Served (Weighted):** For each charger, create a 500m buffer; for each barrio, compute the % of its area covered by any buffer (max across chargers) and count `EV_Count` weighted by that %.
*   **Unmet Demand Covered (Weighted):** Same coverage logic applied to `Unmet_Demand`.
*   **Coverage Efficiency:** `ev_population_served / total_city_evs`.
*   **Efficiency Gain:** % Improvement of Smart over Baseline.

In [15]:
import pandas as pdimport geopandas as gpdimport numpy as npfrom sklearn.cluster import KMeansfrom shapely.geometry import Pointimport randomimport matplotlib.pyplot as pltimport os# 1. Load DataDATA_PATH = '../data/processed/barrios_with_demand.geojson'if not os.path.exists(DATA_PATH):    DATA_PATH = 'data/processed/barrios_with_demand.geojson'gdf = gpd.read_file(DATA_PATH)# Calculate Centroids with proper CRS transformation (Fix for CRS warning)# EPSG:25831 is ETRS89 / UTM zone 31N (Barcelona, Spain) - uses meters# This ensures accurate centroid calculations, then we convert back to lat/lon# Convert to projected CRS (EPSG:25831) for accurate centroid calculationgdf_projected = gdf.to_crs('EPSG:25831')# Calculate centroids in projected CRScentroids_projected = gdf_projected.geometry.centroid# Convert centroids back to EPSG:4326 (lat/lon) for use in algorithmscentroids_gdf = gpd.GeoDataFrame(geometry=centroids_projected, crs='EPSG:25831')centroids_4326 = centroids_gdf.to_crs('EPSG:4326')# Extract lat/lng coordinatesgdf['lat'] = centroids_4326.geometry.ygdf['lng'] = centroids_4326.geometry.x# Recalculate Unmet Demand logic here to be self-containedfrom sklearn.preprocessing import MinMaxScalerscaler = MinMaxScaler()gdf['Norm_Supply'] = scaler.fit_transform(gdf[['Charger_Count']].fillna(0))SUPPLY_IMPACT = 80 gdf['Unmet_Demand'] = gdf['Demand_Score'] - (gdf['Norm_Supply'] * SUPPLY_IMPACT)gdf['Unmet_Demand'] = gdf['Unmet_Demand'].clip(lower=0)print(f"Loaded {len(gdf)} neighborhoods.")

Loaded 73 neighborhoods.


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_92080\1666039808.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_92080\1666039808.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_92080\1666039808.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lng'] = gdf.centroid.x


## Define Simulation Functions

In [16]:
def generate_population_weighted_locations(n, gdf, rng=None, py_rng=None):
    """
    Generates N locations distributed proportionally to Vehicle Density.
    This mimics a 'Standard Policy' approach (more cars = more chargers).

    Parameters
    - rng: numpy Generator (for reproducible barrio sampling)
    - py_rng: python random.Random (for reproducible point placement inside polygons)
    """
    if rng is None:
        rng = np.random.default_rng()
    if py_rng is None:
        py_rng = random.Random()

    points = []
    
    # 1. Select Neighborhoods weighted by Total Vehicles
    # Handle NaNs in weights
    weights = gdf['Total_Vehicles'].fillna(0)
    # Normalize weights to sum to 1
    weights = weights / weights.sum()
    
    # Sample n neighborhoods (indexes) with replacement
    sampled_indices = rng.choice(gdf.index.to_numpy(), size=n, replace=True, p=weights.to_numpy())
    
    # 2. Place a random point in each selected neighborhood
    for idx in sampled_indices:
        poly = gdf.loc[idx, 'geometry']
        min_x, min_y, max_x, max_y = poly.bounds
        
        while True:
            rand_x = py_rng.uniform(min_x, max_x)
            rand_y = py_rng.uniform(min_y, max_y)
            p = Point(rand_x, rand_y)
            if poly.contains(p):
                points.append([p.y, p.x]) # Lat, Lng
                break
                
    return np.array(points)

def generate_smart_locations(n, df_features, weights):
    """Runs Weighted K-Means to find optimal locations."""
    # Safety Check: Cannot request more clusters than data points (neighborhoods)
    n_samples = len(df_features)
    n_clusters = n
    if n > n_samples:
        print(f"Warning: Requested {n} hubs but only {n_samples} neighborhoods available. Capping optimization at {n_samples}.")
        n_clusters = n_samples
        
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    kmeans.fit(df_features, sample_weight=weights)
    return kmeans.cluster_centers_

def evaluate_coverage(locations, gdf_target, radius_m=500):
    """
    Improved evaluation using *EVs only* and a realistic coverage radius.

    What this does (high level):
    - Creates a buffer (default 500m) around each proposed charger location
    - Finds which barrios (neighborhood polygons) intersect each buffer
    - Computes the % of each barrio's area covered by the buffer
    - Counts EVs proportionally to that coverage %
    - De-duplicates barrios across multiple chargers (keeps the best coverage %)

    Notes:
    - We reproject geometries to a metric CRS (EPSG:3857) so that the buffer
      radius is in meters. No extra libraries required beyond GeoPandas/Shapely.
    - `radius_m` is configurable to support sensitivity analysis.
    """
    if len(locations) == 0:
        total_evs_city = float(gdf_target['EV_Count'].fillna(0).sum())
        return {
            'served_barrios_count': 0,
            'ev_population_served': 0.0,
            'unmet_demand_captured': 0.0,
            'coverage_efficiency': 0.0 if total_evs_city == 0 else 0.0
        }

    # --- 1) Prepare barrio polygons in a metric CRS (meters) ---
    barrios = gdf_target[['Barri_ID', 'EV_Count', 'Unmet_Demand', 'geometry']].copy()
    # EPSG:3857 is a common meter-based CRS suitable for ~city-scale buffers.
    barrios_m = barrios.to_crs(epsg=3857)
    barrio_area = barrios_m.geometry.area
    # Avoid division by zero (shouldn't happen, but keep robust).
    barrio_area = barrio_area.replace(0, np.nan)

    # --- 2) Build point buffers around charger locations (also in meters) ---
    loc_df = pd.DataFrame(locations, columns=['lat', 'lng'])
    loc_gdf = gpd.GeoDataFrame(
        loc_df,
        geometry=gpd.points_from_xy(loc_df.lng, loc_df.lat),
        crs=gdf_target.crs
    )
    loc_m = loc_gdf.to_crs(epsg=3857)
    buffers = loc_m.geometry.buffer(radius_m)

    # --- 3) Compute best (max) coverage % per barrio across all buffers ---
    max_coverage = np.zeros(len(barrios_m), dtype=float)

    # With ~73 barrios and <=50 chargers, this loop is fast and easy to reason about.
    for buf in buffers:
        # Intersection area for every barrio with this buffer
        inter_area = barrios_m.geometry.intersection(buf).area
        coverage_pct = (inter_area / barrio_area).fillna(0).clip(lower=0, upper=1).to_numpy()
        max_coverage = np.maximum(max_coverage, coverage_pct)

    served_mask = max_coverage > 0
    served_barrios_count = int(served_mask.sum())

    # --- 4) Compute EVs served (weighted by coverage %) ---
    ev_counts = barrios['EV_Count'].fillna(0).to_numpy(dtype=float)
    ev_population_served = float((ev_counts * max_coverage).sum())

    # Keep Unmet Demand logic consistent with the new coverage model by weighting it too
    # (partial barrio coverage captures partial unmet demand).
    unmet_vals = barrios['Unmet_Demand'].fillna(0).to_numpy(dtype=float)
    unmet_demand_captured = float((unmet_vals * max_coverage).sum())

    # --- 5) Coverage efficiency: EVs served / total EVs in the city ---
    total_evs_city = float(barrios['EV_Count'].fillna(0).sum())
    coverage_efficiency = 0.0 if total_evs_city == 0 else float(ev_population_served / total_evs_city)

    return {
        'served_barrios_count': served_barrios_count,
        'ev_population_served': ev_population_served,
        'unmet_demand_captured': unmet_demand_captured,
        'coverage_efficiency': coverage_efficiency
    }

## Run Simulation Loop

In [17]:
# Config
SCENARIOS = [10, 25, 50] # Adjusted to stay safely under 73 limit

# Baseline is stochastic (random). To make results reliable, we run multiple
# iterations and summarize the distribution (mean + percentiles).
BASELINE_ITERATIONS = 300
BASELINE_SEED = 42

# Coverage radius (meters) used by evaluate_coverage()
RADIUS_M = 500

results = []
locations_export = []

X_coords = gdf[['lat', 'lng']].values
W_weights = gdf['Unmet_Demand'].fillna(0).values

def _pct(arr, q):
    """Helper for percentiles with numpy arrays."""
    return float(np.percentile(arr, q)) if len(arr) else float('nan')

for n in SCENARIOS:
    print(f"--- Simulating N={n} --- ")
    
    # ------------------------------------------------------------
    # 1) Baseline (Population Weighted) with ITERATIONS
    # ------------------------------------------------------------
    # Export ONE representative baseline layout for mapping (stable seed).
    rng_map = np.random.default_rng(BASELINE_SEED + n)
    py_rng_map = random.Random(BASELINE_SEED + n)
    base_locs = generate_population_weighted_locations(n, gdf, rng=rng_map, py_rng=py_rng_map)
    
    # Run many baseline iterations for robust KPIs
    baseline_metrics = []
    for it in range(BASELINE_ITERATIONS):
        # Different seeds per iteration for reproducible stochastic baseline distribution
        seed_it = (BASELINE_SEED * 1_000_000) + (n * 10_000) + it
        rng_it = np.random.default_rng(seed_it)
        py_rng_it = random.Random(seed_it)
        locs_it = generate_population_weighted_locations(n, gdf, rng=rng_it, py_rng=py_rng_it)
        baseline_metrics.append(evaluate_coverage(locs_it, gdf, radius_m=RADIUS_M))

    # Convert baseline distribution to arrays
    base_served = np.array([m['served_barrios_count'] for m in baseline_metrics], dtype=float)
    base_ev = np.array([m['ev_population_served'] for m in baseline_metrics], dtype=float)
    base_unmet = np.array([m['unmet_demand_captured'] for m in baseline_metrics], dtype=float)
    base_eff = np.array([m['coverage_efficiency'] for m in baseline_metrics], dtype=float)

    base_summary = {
        # Use MEAN as the main baseline KPI point estimate
        'served_barrios_count': float(base_served.mean()),
        'ev_population_served': float(base_ev.mean()),
        'unmet_demand_captured': float(base_unmet.mean()),
        'coverage_efficiency': float(base_eff.mean()),
        # Extra distribution columns (useful for Tableau bands / uncertainty)
        'baseline_iterations': int(BASELINE_ITERATIONS),
        'ev_population_served_p05': _pct(base_ev, 5),
        'ev_population_served_p50': _pct(base_ev, 50),
        'ev_population_served_p95': _pct(base_ev, 95),
        'unmet_demand_captured_p05': _pct(base_unmet, 5),
        'unmet_demand_captured_p50': _pct(base_unmet, 50),
        'unmet_demand_captured_p95': _pct(base_unmet, 95),
        'coverage_efficiency_p05': _pct(base_eff, 5),
        'coverage_efficiency_p50': _pct(base_eff, 50),
        'coverage_efficiency_p95': _pct(base_eff, 95),
    }

    results.append({
        'N_Chargers': n,
        'Strategy': 'Baseline',
        **base_summary
    })
    
    # Add representative Baseline locations to Export List (for maps)
    for i, loc in enumerate(base_locs):
        locations_export.append({
            'Scenario_ID': f'Baseline_{n}',
            'Type': 'Baseline',
            'N_Chargers': n,
            'Lat': loc[0],
            'Lng': loc[1],
            'Hub_ID': i+1
        })

    # ------------------------------------------------------------
    # 2) Smart (Unmet Demand) - deterministic (random_state fixed)
    # ------------------------------------------------------------
    smart_locs = generate_smart_locations(n, X_coords, W_weights)
    smart_metrics = evaluate_coverage(smart_locs, gdf, radius_m=RADIUS_M)

    # How often does Smart beat a random Baseline draw?
    ev_win_rate = float((smart_metrics['ev_population_served'] > base_ev).mean())
    unmet_win_rate = float((smart_metrics['unmet_demand_captured'] > base_unmet).mean())
    eff_win_rate = float((smart_metrics['coverage_efficiency'] > base_eff).mean())

    results.append({
        'N_Chargers': n,
        'Strategy': 'Smart',
        **smart_metrics,
        'baseline_iterations': int(BASELINE_ITERATIONS),
        'ev_win_rate_vs_baseline': ev_win_rate,
        'unmet_win_rate_vs_baseline': unmet_win_rate,
        'eff_win_rate_vs_baseline': eff_win_rate,
        'baseline_ev_mean': float(base_ev.mean()),
        'baseline_ev_p05': _pct(base_ev, 5),
        'baseline_ev_p50': _pct(base_ev, 50),
        'baseline_ev_p95': _pct(base_ev, 95),
    })
    
    # Add Smart locations to Export List
    for i, loc in enumerate(smart_locs):
        locations_export.append({
            'Scenario_ID': f'Smart_{n}',
            'Type': 'Smart',
            'N_Chargers': n,
            'Lat': loc[0],
            'Lng': loc[1],
            'Hub_ID': i+1
        })
    
print("Simulation Complete.")

--- Simulating N=10 --- 
--- Simulating N=25 --- 
--- Simulating N=50 --- 
Simulation Complete.


## Analysis and KPI Calculation

In [18]:
df_res = pd.DataFrame(results)

# Pivot to compare side-by-side
df_pivot = df_res.pivot(index='N_Chargers', columns='Strategy', values=['ev_population_served', 'unmet_demand_captured', 'coverage_efficiency'])

# Calculate Efficiency Gains
df_pivot['EV_Pop_Gain_Pct'] = (df_pivot[('ev_population_served', 'Smart')] - df_pivot[('ev_population_served', 'Baseline')]) / df_pivot[('ev_population_served', 'Baseline')] * 100
df_pivot['Demand_Gain_Pct'] = (df_pivot[('unmet_demand_captured', 'Smart')] - df_pivot[('unmet_demand_captured', 'Baseline')]) / df_pivot[('unmet_demand_captured', 'Baseline')] * 100

display(df_pivot)

# Save Results
# 1. Scenarios File
df_locs_export = pd.DataFrame(locations_export)
df_locs_export.to_csv('../data/processed/tableau_scenarios.csv', index=False)

# 2. KPIs File
df_kpis = df_res.copy()
# Just simple flattening for Tableau
df_kpis['Scenario_ID'] = df_kpis['Strategy'] + '_' + df_kpis['N_Chargers'].astype(str)
df_kpis.to_csv('../data/processed/tableau_kpis.csv', index=False)

# 3. Master Barrio File (ensure it has metrics)
gdf_export = gdf.drop(columns=['geometry', 'centroid'], errors='ignore') # simple CSV for data attributes
gdf_export.to_csv('../data/processed/tableau_barrios_master.csv', index=False)

print("Exported CSVs for Tableau.")

ev_population_served               unmet_demand_captured  \
Strategy               Baseline         Smart              Baseline   
N_Chargers                                                            
10                  4465.551693   4991.302048             95.843969   
25                  9187.245127  13399.695157            199.763724   
50                 14245.151587  22575.719942            311.278521   

                       coverage_efficiency           EV_Pop_Gain_Pct  \
Strategy         Smart            Baseline     Smart                   
N_Chargers                                                             
10          118.285899            0.059217  0.066189       11.773469   
25          311.259376            0.121831  0.177691       45.851068   
50          536.529363            0.188903  0.299373       58.480026   

           Demand_Gain_Pct  
Strategy                    
N_Chargers                  
10               23.415068  
25               55.813763  
50               72.363118

Exported CSVs for Tableau.
